In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints, pca
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints
from mtflearn.utils import normalize_image, remove_bg
from mtflearn.manifold import ForceGraph8
from mtflearn.denoise import denoise_svd

import re
from skimage.transform import resize
from stemplot.utils import plot_pca
from stemplot.interactive import interactive_clusters
from stemplot.colors import generate_colors_from_lbs,color_palette

In [4]:
img = load_image('0106_ADF2_12.dm4')
img = normalize_image(img)
lbs = np.load('Ti 2 lbs.npy').astype(int)
pts = np.load('Ti 2 pts.npy')

In [5]:
mask1 = np.logical_or(lbs==0, lbs==1)
mask2 = np.logical_or(lbs==2, lbs==3)
img = resize(img, (1024, 1024), order=0)
size = 36

In [6]:
kp = KeyPoints(pts, img, size)
ps = kp.extract_patches(size)

In [7]:
zps = ZPs(n_max=12, size=ps.shape[1])
moments = zps.fit_transform(ps)
X = moments.select([0]).data
#X1 = X[mask1]
X1 = X[mask1][:, 1:]
X2 = X[mask2][:, 1:]
lbs1 = lbs[mask1].astype(int)
lbs2 = lbs[mask2].astype(int)

In [8]:
fg = ForceGraph8(X = X1,
                 n_neighbors=5,
                 init_mode = 'pca',
                 num_iterations = 100,
                 num_negative_samples=5,
                 force_params1=(0, 2, 1, 1),
                 force_params2=(2, 5, 5, 1),
                 divide=0.5)

from time import time
t0 = time()
xy1 = fg.fit_transform(X1)

#app1 = interactive_clusters(pca(X1), img2, pts[mask1], ps[mask1], lbs=lbs1, s=1, max_samples=10000) 
app1 = interactive_clusters(xy1, img, pts[mask1], ps[mask1], lbs=lbs1, s=1, max_samples=10000)

In [18]:
fig,ax = plt.subplots(figsize = (8,8))
colors = generate_colors_from_lbs(lbs1)
ax.scatter(xy1[:,0],xy1[:,1],s = 5, c=colors)
ax.set_xlabel('FR Component 1')
ax.set_ylabel('FR Component 2')
plt.savefig('Se center layout.png',dpi = 700,transparent=True)

In [12]:
Se_ps = ps[lbs==0]
V_ps = ps[lbs==1]

In [13]:
fig,ax = plt.subplots(figsize = (8,8))
ax.imshow(np.mean(Se_ps,axis=0),cmap = color_palette('C0'))
plt.axis('off')
plt.savefig('Se2 center mean',dpi = 700,transparent=True)

In [14]:
fig,ax = plt.subplots(figsize = (8,8))
ax.imshow(np.mean(V_ps,axis=0),cmap = color_palette('C1'))
plt.axis('off')
plt.savefig('V center mean',dpi = 700,transparent=True)

### Metal part

In [15]:
xy2 = pca(X2)
app2 = interactive_clusters(pca(X2), img, pts[mask2], ps[mask2], lbs=lbs2, s=1)

In [16]:
X2.max(),X2.min()

In [19]:
fig,ax = plt.subplots(figsize = (8,8))
colors = generate_colors_from_lbs(lbs2)
ax.scatter(xy2[:,0],xy2[:,1],s = 5, c=colors)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
plt.savefig('W dope Ti center layout.png',dpi = 700,transparent=True)

In [20]:
W_ps = ps[lbs==2]
D_ps = ps[lbs==3]

In [21]:
fig,ax = plt.subplots(figsize = (8,8))
ax.imshow(np.mean(W_ps,axis=0),cmap = color_palette('C2'))
plt.axis('off')
plt.savefig('W center mean',dpi = 700,transparent=True)

In [22]:
fig,ax = plt.subplots(figsize = (8,8))
ax.imshow(np.mean(D_ps,axis=0),cmap = color_palette('C3'))
plt.axis('off')
plt.savefig('Ti center mean',dpi = 700,transparent=True)

In [104]:
select_pts = np.vstack([pts[lbs==1],pts[lbs==3]])
select_lbs = np.hstack([lbs[lbs==1],lbs[lbs==3]])
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img, cmap='gray')
ax.scatter(select_pts[:, 0], select_pts[:, 1], color=colors_from_lbs(select_lbs), s=10)
ax.axis('off')
# plt.savefig('points on image.png',dpi = 700,transparent=True)